This notebook aims at computing a precise estimate of the dynamic parameters of the equivalent circuit model, namely R0, R1 and C1. The data used to give that estimation must satisfy a few criteria : 
- Battery must have been at rest before logging start, which means that the parameter allowing logging to start should be set in QGC.
- The log should cover the entirecharge range of the battery, i.e. the battery should go form 100% charge to 0% during the log, so that the different battery behaviors at all states of charge are captured.
- If the estimation is performed on several logs, all logs should come from the same battery with similar age. 
- Total capacity Q and coulombic efficiency eta (statis parameters) should be computed on that very same battery, in the same condition as the dynamic tests.


In [1]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append('/home/lucas/Documents/Log_Analysis/Battery')
import analog
import battery
import matplotlib.pyplot as plt
%matplotlib notebook


In [2]:
curve = battery.OCVcurve('SOCvsOCV_discharge200mA.csv')
z0 = 1
Q = 6000*3.6

In [3]:
folder = '/home/lucas/Documents/Log_Analysis/Logs/Jack Sparrow (Luigi) z0=1'
files = os.listdir(folder)

time = np.array([])
current = np.array([])
voltage = np.array([])
tstoplist = [0]
for file in sorted(files):
    print(file)
    info = analog.logextract(f'{folder}/{file}','battery_status')
    time = np.append(time,info['time_bs']- info['time_bs'][0] + tstoplist[-1])
    tstoplist.append(time[-1])
    current = np.append(current,info['battery_current'])
    voltage = np.append(voltage,info['battery_voltage']/4)

log_65_2019-9-24-14-46-00.ulg
log_66_2019-9-24-14-47-42.ulg
log_67_2019-9-24-14-54-06.ulg
log_68_2019-9-24-14-58-52.ulg
log_69_2019-9-24-15-01-40.ulg
log_70_2019-9-24-15-08-10.ulg
log_71_2019-9-24-15-14-24.ulg


In [4]:
%matplotlib notebook
plt.figure()
plt.subplot(211)
plt.plot(time,voltage)
for tstop in tstoplist:
    plt.axvline(tstop,color='r',linestyle='-.',linewidth=0.7)
plt.ylabel('terminal voltage (V)')
plt.title('Input/Output time evolution of the battery ')
plt.grid()
plt.subplot(212)
plt.plot(time,current)
plt.xlabel('time (s)')
plt.ylabel('current (A)')
plt.grid()
plt.show()

<IPython.core.display.Javascript object>

In [5]:
from scipy import optimize
def error(param):
    optbattery = battery.Thevenin(z0,Q,curve,param[0],param[1],param[2])
    vsim = optbattery.simulate(time,current,optbattery.OCVcurve)
    rmserror = (np.mean((vsim-voltage)**2))**.5
    return rmserror

bounds = [(1e-5,1e-4),(1e-5,.1),(1e-5,10e3)]
results = optimize.basinhopping(error,bounds)
# results = optimize.differential_evolution(error,bounds)
print(results)

/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: invalid value encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/scipy/optimize/optimize.py:696: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/home/lucas/Documents/Log_Analysis/Battery/battery.py:245: RuntimeWarning: overflow encountered in double_scalars
  self.simv.append(OCVcurve.OCVfromSOC

/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: invalid value encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: over

/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: invalid value encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: over

/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/scipy/optimize/optimize.py:696: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/Doc

/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/scipy/optimize/optimize.py:696: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/home/lucas/Doc

/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/scipy/optimize/optimize.py:696: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/Doc

/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/scipy/optimize/optimize.py:696: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/Doc

/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/scipy/optimize/optimize.py:696: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: invalid value encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/luca

/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """
/home/lucas/anaconda3/envs/analysis/lib/python3.7/site-packages/scipy/optimize/optimize.py:696: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/home/lucas/Documents/Log_Analysis/Battery/battery.py:244: RuntimeWarning: overflow encountered in double_scalars
  self.simi1.append(np.exp(-self.simdt/(self.R1*self.C1)) * self.simi1[k] + (1 - np.exp(-self.simdt/(self.R1*self.C1)))*self.simi[k])
/home/lucas/ana

KeyboardInterrupt: 

In [ ]:
optbattery = battery.Thevenin(z0,Q,results.x[0],results.x[1],results.x[2])
vsimopt = optbattery.simulate(time,current,OCVfromSOC)

In [ ]:
error = np.sum((vsimopt - voltage)**2)
print(error)